In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords

from nltk.stem.lancaster import LancasterStemmer #model for stemming words
stemmer = LancasterStemmer()

import gensim
import string
from gensim import corpora
from gensim.corpora.dictionary import Dictionary

%matplotlib inline

In [2]:
df = pd.read_csv('/content/test_data.csv')

In [3]:
#Удалим столбец 'line_n'
df = df.drop(columns=['line_n'])

In [4]:
df.head()

,dlg_id,role,text
0,0,manager,Алло
1,0,client,Алло здравствуйте
2,0,manager,Добрый день
3,0,client,Меня зовут ангелина компания диджитал бизнес з...
4,0,manager,Ага


In [5]:
#Приведем текст к нижнему регистру, удалим символы и цифры.
def text_clean(df):
  df['text'] = df['text'].str.lower().replace('ё', 'е')
  df['text'] = df['text'].str.replace(r'[0-9]', '')
  df['text'] = df['text'].str.replace(r'[^a-zA-Zа-яА-Я1-9]+', ' ')

In [6]:
text_clean(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [7]:
#Определим токенизатор
tokenizer = RegexpTokenizer(r'\w+')

In [8]:
#Создадим новый столбец с токенами
df['tokenized_text'] = df['text'].apply(tokenizer.tokenize)
df.head()

,dlg_id,role,text,tokenized_text
0,0,manager,алло,[алло]
1,0,client,алло здравствуйте,"[алло, здравствуйте]"
2,0,manager,добрый день,"[добрый, день]"
3,0,client,меня зовут ангелина компания диджитал бизнес з...,"[меня, зовут, ангелина, компания, диджитал, би..."
4,0,manager,ага,[ага]


In [9]:
df.loc[[146]]

,dlg_id,role,text,tokenized_text
146,1,client,правильно,[правильно]


In [10]:
#Создадим словарь
Vocab = []
for i in df.text:
  Vocab.append(i)

In [11]:
Vocab = tokenizer.tokenize(str(Vocab))

In [12]:
print(Vocab)

['алло', 'алло', 'здравствуйте', 'добрый', 'день', 'меня', 'зовут', 'ангелина', 'компания', 'диджитал', 'бизнес', 'звоним', 'вам', 'по', 'поводу', 'продления', 'лицензии', 'а', 'мы', 'с', 'серым', 'у', 'вас', 'скоро', 'срок', 'заканчивается', 'ага', 'угу', 'ну', 'возможно', 'вы', 'рассмотрите', 'и', 'другие', 'варианты', 'видите', 'это', 'хорошая', 'практика', 'сравнивать', 'да', 'мы', 'работаем', 'с', 'компанией', 'которая', 'нам', 'подливает', 'поэтому', 'спасибо', 'огромное', 'как', 'как', 'бы', 'уже', 'до', 'этого', 'момента', 'работаем', 'все', 'устраивает', 'у', 'нас', 'сопровождение', 'поэтому', 'угу', 'а', 'на', 'что', 'вы', 'обращаете', 'внимание', 'при', 'выборе', 'как', 'бы', 'нет', 'да', 'на', 'выбор', 'я', 'уже', 'не', 'обращаю', 'внимание', 'я', 'уже', 'дома', 'выбрал', 'что', 'для', 'вас', 'приоритет', 'поэтому', 'типа', 'много', 'параметров', 'выбор', 'не', 'выбор', 'есть', 'история', 'сотрудничества', 'которая', 'более', 'важна', 'потому', 'что', 'вот', 'смотрите', 'вы

In [13]:
#Проверим, какие слова встречаются чаще других
from nltk.probability import FreqDist

fdist = FreqDist(Vocab)
print(fdist)

<FreqDist with 1142 samples and 4634 outcomes>


In [14]:
fdist.most_common(10)

[('да', 148),
 ('вот', 127),
 ('то', 112),
 ('и', 108),
 ('ну', 93),
 ('а', 87),
 ('я', 84),
 ('вы', 81),
 ('что', 78),
 ('мы', 74)]

In [15]:
#Чаще других встречаются слова "вот" "то" "да" "ну" "а". Удалим ненужные слова
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('russian'))
print(stop_words)

{'но', 'над', 'ли', 'почти', 'этого', 'они', 'под', 'меня', 'его', 'про', 'всего', 'ж', 'так', 'после', 'в', 'во', 'ведь', 'до', 'чтобы', 'если', 'этот', 'не', 'был', 'потом', 'будто', 'надо', 'за', 'всю', 'их', 'лучше', 'ним', 'со', 'этой', 'чуть', 'более', 'что', 'мне', 'разве', 'на', 'сам', 'нее', 'уже', 'же', 'моя', 'для', 'без', 'когда', 'раз', 'вас', 'еще', 'него', 'чтоб', 'я', 'вы', 'или', 'может', 'было', 'другой', 'а', 'куда', 'теперь', 'впрочем', 'он', 'да', 'ну', 'то', 'у', 'и', 'себя', 'ты', 'нельзя', 'зачем', 'здесь', 'где', 'с', 'по', 'через', 'ней', 'мой', 'чем', 'нас', 'какой', 'вам', 'между', 'ей', 'нет', 'были', 'больше', 'мы', 'конечно', 'бы', 'как', 'всех', 'чего', 'два', 'хоть', 'можно', 'тебя', 'том', 'много', 'эту', 'того', 'там', 'какая', 'такой', 'тот', 'опять', 'нибудь', 'тогда', 'тем', 'есть', 'эти', 'была', 'о', 'тут', 'ее', 'уж', 'будет', 'к', 'ни', 'кто', 'при', 'перед', 'свою', 'ничего', 'тоже', 'три', 'все', 'этом', 'потому', 'никогда', 'только', 'себе',

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
#Словарь без "стоп-слов"
clean_Vocab = []

for w in Vocab:
  if w not in stop_words:
    clean_Vocab.append(w)

In [17]:
fdist = FreqDist(clean_Vocab)
fdist.most_common(10)

[('это', 72),
 ('угу', 49),
 ('просто', 30),
 ('например', 24),
 ('смотрите', 21),
 ('нам', 19),
 ('правильно', 17),
 ('давайте', 16),
 ('вами', 15),
 ('бизнес', 14)]

In [18]:
#Поиск синонимов
import gensim
from gensim.models import Word2Vec

In [19]:
model = Word2Vec([clean_Vocab], min_count =3)

In [20]:
model.train([clean_Vocab], total_examples=model.corpus_count, epochs=200, report_delay=1) 

(123759, 414800)

In [21]:
#Синонимы "здравствуйте"
model.most_similar('здравствуйте')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('зовут', 0.9827433824539185),
 ('ангелина', 0.9585208892822266),
 ('день', 0.9443455934524536),
 ('диджитал', 0.9185293912887573),
 ('добрый', 0.9154111742973328),
 ('алло', 0.8987537622451782),
 ('свидания', 0.8947328925132751),
 ('удобно', 0.8937771320343018),
 ('дмитрий', 0.8899378776550293),
 ('хорошего', 0.8867058753967285)]

In [22]:
#Синонимы "до свидания"
model.most_similar('свидания')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('алло', 0.9440104961395264),
 ('дмитрий', 0.9384851455688477),
 ('добрый', 0.9208201766014099),
 ('хорошего', 0.9136171340942383),
 ('день', 0.9020993709564209),
 ('здравствуйте', 0.8947328925132751),
 ('понедельник', 0.8560622334480286),
 ('зовут', 0.8553675413131714),
 ('удобно', 0.8548212051391602),
 ('связь', 0.8392418622970581)]

Реплики с приветствием – где менеджер поздоровался

In [23]:
#Новый датасет для извлечения информации о менеджере
df_manager = df.loc[(df['role'] == 'manager')]

In [24]:
df_manager.head()

,dlg_id,role,text,tokenized_text
0,0,manager,алло,[алло]
2,0,manager,добрый день,"[добрый, день]"
4,0,manager,ага,[ага]
6,0,manager,да мы работаем с компанией которая нам подлива...,"[да, мы, работаем, с, компанией, которая, нам,..."
7,0,manager,как как бы уже до этого момента работаем все у...,"[как, как, бы, уже, до, этого, момента, работа..."


In [25]:
import re as re

In [26]:
def get_info(str, word):
    result = re.findall(r'([^.]*'+word+'[^.]*)', str)
    return result

In [27]:
df_manager['greetings']=df_manager['text'].apply(lambda x : get_info(x,'здравствуйте'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Реплики, где менеджер представил себя

In [28]:
df_manager['introduction']=df_manager['text'].apply(lambda x : get_info(x,'зовут'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Реплики, где менеджер попрощался

In [29]:
df_manager['bye']=df_manager['text'].apply(lambda x : get_info(x,'свидания'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Имя менеджера и организации. К сожалению, я не смогла выполнить это задание.

In [30]:
pip install spacy-stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import stanza
nlp = stanza.Pipeline('ru')

docs = []

for sentence in df.text:
  doc = nlp(sentence)
  docs.append(docs)

for ent in docs.sentences[0].entities:
    if (ent.type in ('PER', 'ORG')):
        print (ent.text,' ',ent.type)

INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
